# Assignment 6
## Project 1
The first project you can choose is a continuation of the InterPROscan dataset you worked on for Assignment 5. You now know the structure and type of data that is involved. What I would ike you to do is to develop scikit-learn machine learning models to predict the function of the proteins in the dataset. I'm not going to mandate any particular model, but you should use your knowledge from DS3 to determine good candidate methods and try a couple, and report on yourGiven the size of the data, you should train your machine models in parallel! Dask is an obvious candidate for this, but you can also use any of the other frameworks we discussed in the course.

The "function" of the protein which is the "class" your model should predict is defined as the InterPRO number which covers:

1. more than 90% of the protein's sequence
2. Covers the largest length of the sequence

The "features" that you use to predict that class are the other, smaller, InterPRO annotations. Take care to remove "noise" from the dataset, ie. lines from the TSV file which don't contain InterPRO numbers, proteins which don't have a large feature (according to the criteria above) etc. Be sure to note in your report how you processed the data and why you made the choices you did!

Your script should, given one or more of these TSV files with InterPROscan annotations, produce models (Pickle them to save them!) which predict protein function, and a file of training-data on which you trained them.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
#path = "/data/dataprocessing/interproscan/all_bacilli.tsv"
